In [1]:
'''
2014年7月4日人工智能实训作业。
Task 1: 基于下面的代码实现训练penn数据集(可以取部分数据实现即可) ，并计算测试集PPL值。(注意：需要更改Postion Emb部分)
Task 2: 实现Beam Search解码方法
完成之后，请把代码和训练日志提交到: https://send2me.cn/DtYAmnVi/QJ68hwswVdVgDg
'''


'\n2014年7月4日人工智能实训作业。\nTask 1: 基于下面的代码实现训练penn数据集(可以取部分数据实现即可) ，并计算测试集PPL值。(注意：需要更改Postion Emb部分)\nTask 2: 实现Beam Search解码方法\n完成之后，请把代码和训练日志提交到: https://send2me.cn/DtYAmnVi/QJ68hwswVdVgDg\n'

In [2]:
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F


In [118]:
src_len = 5 # length of source
tgt_len = 5 # length of target

d_model = 128  # Embedding Size
d_ff = 512  # FeedForward dimension
d_k = d_v = 64  # dimension of K(=Q), V
n_layers = 6  # number of Encoder of Decoder Layer
n_heads = 8  # number of heads in Multi-Head Attention

batch_size = 512

train_path = './data/train.txt' # the path of train dataset
valid_path = './data/valid.txt'
test_psth = './data/test.txt'

src_vocab , _ = make_dict(train_path)
src_vocab_size = len(src_vocab)
tgt_vocab , number_dict = make_dict(train_path)
tgt_vocab_size = len(tgt_vocab)

def make_dict(path):
    with open(path, 'r', encoding='utf-8') as fr:
        text = fr.readlines()
    text = [n.strip() for n in text]
    text = ' '.join(text).split(" ")
    text = list(set(text))
    
    word2number_dict = {}
    number2word_dict = {}
    for i,word in enumerate(text,4):
        word2number_dict[word] = i
        number2word_dict[i] = word
    word2number_dict["<pad>"] = 0
    number2word_dict[0] = "<pad>"
    word2number_dict["<unk_word>"] = 1
    number2word_dict[1] = "<unk_word>"
    word2number_dict["<sos>"] = 2
    number2word_dict[2] = "<sos>"
    word2number_dict["<eos>"] = 3
    number2word_dict[3] = "<eos>"

    return word2number_dict, number2word_dict
def make_batch(path,src_vocab,tgt_vocab,batch_size):
    def src_word2number(word):
        try:
            return src_vocab[word]
        except :
            return 1
    def tgt_word2number(word):
        try:
            return tgt_vocab[word]
        except :
            return 1
    input_batchs = []
    output_batchs = []
    target_batchs = []
    with open(path,'r',encoding='utf-8') as f:
        text = f.readlines()
    text =[line.strip() for line in text]
    
    input_batch = []
    output_batch = []
    target_batch = []
    
    for sentences in text:
        
        words = sentences.split()
        
        if len(words) <src_len+tgt_len:continue
        
        for i in range(len(words)-src_len-tgt_len+1):
            
            input_batch.append([src_word2number(words[i+k]) for k in range(src_len)])
            
            output = [tgt_word2number(words[i+src_len+k]) for k in range(tgt_len)]*2
            
            output.insert(0,tgt_word2number('<sos>'))
            output.append(tgt_word2number('<eos>'))
            
            output_batch.append(output[0:tgt_len+1])
            target_batch.append(output[tgt_len+1:])
            
    for i in range(0,len(input_batch),batch_size):
        if i+batch_size>len(input_batch):
            break
        input_batchs.append(input_batch[i:i+batch_size])
        output_batchs.append(output_batch[i:i+batch_size])
        target_batchs.append(target_batch[i:i+batch_size])
    
    return torch.LongTensor(input_batchs), torch.LongTensor(output_batchs), torch.LongTensor(target_batchs)

class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
        self.linear = nn.Linear(n_heads * d_v, d_model)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]

        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size x n_heads x len_q x len_k]

        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = self.linear(context)
        return self.layer_norm(output + residual), attn # output: [batch_size x len_q x d_model]

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, inputs):
        residual = inputs # inputs : [batch_size, len_q, d_model]
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        output = self.conv2(output).transpose(1, 2)
        return self.layer_norm(output + residual)

class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs, attn

class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()
        self.dec_enc_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        dec_outputs, dec_enc_attn = self.dec_enc_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        dec_outputs = self.pos_ffn(dec_outputs)
        return dec_outputs, dec_self_attn, dec_enc_attn

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(src_vocab_size, d_model)
        self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(src_len+1, d_model),freeze=True)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs): # enc_inputs : [batch_size x source_len]
        
        enc_outputs = self.src_emb(enc_inputs) + self.pos_emb(abs_position_encode(enc_inputs))# pos_emb出错
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        enc_self_attns = []
        for layer in self.layers:
            enc_outputs, enc_self_attn = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attns.append(enc_self_attn)
        return enc_outputs, enc_self_attns

def abs_position_encode(inputs):
    batch_size , seq_len = inputs.size()
    position_encode = []
    for i in range(batch_size):
        position_encode.append([k for k in range(seq_len)])
    return torch.LongTensor(position_encode)

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(tgt_vocab_size, d_model)
        self.pos_emb = nn.Embedding.from_pretrained(get_sinusoid_encoding_table(tgt_len+1, d_model),freeze=True)
        self.layers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)])

    def forward(self, dec_inputs, enc_inputs, enc_outputs): # dec_inputs : [batch_size x target_len]
        dec_outputs = self.tgt_emb(dec_inputs) + self.pos_emb(abs_position_encode(dec_inputs))
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)
        dec_self_attn_subsequent_mask = get_attn_subsequent_mask(dec_inputs)
        dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequent_mask), 0)

        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)

        dec_self_attns, dec_enc_attns = [], []
        for layer in self.layers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attns.append(dec_self_attn)
            dec_enc_attns.append(dec_enc_attn)
        return dec_outputs, dec_self_attns, dec_enc_attns

class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
        self.projection = nn.Linear(d_model, tgt_vocab_size, bias=False)
    def forward(self, enc_inputs, dec_inputs):
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        dec_logits = self.projection(dec_outputs) # dec_logits : [batch_size x src_vocab_size x tgt_vocab_size]
        return dec_logits.view(-1, dec_logits.size(-1)), enc_self_attns, dec_self_attns, dec_enc_attns


# S: Symbol that shows starting of decoding input
# E: Symbol that shows starting of decoding output
# P: Symbol that will fill in blank sequence if current batch data size is short than time steps

def get_sinusoid_encoding_table(n_position, d_model):
    def cal_angle(position, hid_idx):
        return position / np.power(10000, 2 * (hid_idx // 2) / d_model)
    def get_posi_angle_vec(position):
        return [cal_angle(position, hid_j) for hid_j in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(n_position)])
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1
    return torch.FloatTensor(sinusoid_table)

def get_attn_pad_mask(seq_q, seq_k):
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # batch_size x 1 x len_k(=len_q), one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # batch_size x len_q x len_k

def get_attn_subsequent_mask(seq):
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    subsequent_mask = np.triu(np.ones(attn_shape), k=1)
    subsequent_mask = torch.from_numpy(subsequent_mask).byte()
    return subsequent_mask

In [128]:
from tqdm import tqdm
import math

if __name__ == '__main__':
    
    model = Transformer()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 10

    enc_inputs_train, dec_inputs_train, target_train = make_batch(train_path, src_vocab=src_vocab, tgt_vocab=tgt_vocab, batch_size=512)
    enc_inputs_val, dec_inputs_val, target_val = make_batch(valid_path, src_vocab=src_vocab, tgt_vocab=tgt_vocab, batch_size=128)  # Replace with your validation data
    
    print("Training batch_num: ", len(enc_inputs_train))
    print("Validation batch_num: ", len(enc_inputs_val))
    
    for epoch in range(epochs):
        # Training
        epoch_loss = 0
        model.train()  # Set model to training mode
        with tqdm(total=len(enc_inputs_train), desc=f'Training Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
            for enc_inputs, dec_inputs, target_batch in zip(enc_inputs_train, dec_inputs_train, target_train):
                optimizer.zero_grad()
                outputs, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)
                loss = criterion(outputs, target_batch.contiguous().view(-1))
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()
                pbar.set_postfix({'loss': f'{loss.item():.6f}', 'ppl': f'{math.exp(loss.item()):.6f}'})
                pbar.update(1)
        
        # Calculate average training loss and perplexity
        avg_train_loss = epoch_loss / len(enc_inputs_train)
        avg_train_ppl = math.exp(avg_train_loss)
        
        # Validation
        model.eval()  # Set model to evaluation mode
        val_loss = 0
        with tqdm(total=len(enc_inputs_val), desc=f'Validation Epoch {epoch+1}/{epochs}', unit='batch') as pbar:
            for enc_inputs, dec_inputs, target_batch in zip(enc_inputs_val, dec_inputs_val, target_val):
                with torch.no_grad():  # No need to compute gradients during validation
                    outputs, _, _, _ = model(enc_inputs, dec_inputs)
                    loss = criterion(outputs, target_batch.contiguous().view(-1))
                    val_loss += loss.item()
                    pbar.update(1)
        
        # Calculate average validation loss and perplexity
        avg_val_loss = val_loss / len(enc_inputs_val)
        avg_val_ppl = math.exp(avg_val_loss)
        
        # Print epoch metrics
        print(f'Epoch {epoch+1} completed. Train Loss: {avg_train_loss:.6f}, Train PPL: {avg_train_ppl:.6f}, Val Loss: {avg_val_loss:.6f}, Val PPL: {avg_val_ppl:.6f}')
        
        # Save model checkpoint (optional)
        torch.save(model.state_dict(), f'models/epoch-{epoch+1}.pt')


Training batch_num:  103
Validation batch_num:  29


Validation Epoch 1/10: 100%|██████████| 29/29 [00:04<00:00,  6.36batch/s]


Epoch 1 completed. Train Loss: 6.254008, Train PPL: 520.092937, Val Loss: 5.892387, Val PPL: 362.269071


Validation Epoch 2/10: 100%|██████████| 29/29 [00:04<00:00,  6.32batch/s]


Epoch 2 completed. Train Loss: 5.906627, Train PPL: 367.464770, Val Loss: 5.919498, Val PPL: 372.224773


...

Validation Epoch 6/10: 100%|██████████| 29/29 [00:04<00:00,  6.31batch/s]


Epoch 6 completed. Train Loss: 5.792697, Train PPL: 327.896138, Val Loss: 6.019469, Val PPL: 411.360071


Validation Epoch 7/10: 100%|██████████| 29/29 [00:04<00:00,  6.38batch/s]


Epoch 7 completed. Train Loss: 5.778007, Train PPL: 323.114489, Val Loss: 6.034872, Val PPL: 417.745227


Training Epoch 8/10:  46%|████▌     | 47/103 [01:18<01:33,  1.67s/batch, loss=5.703857, ppl=300.022342]

In [113]:
def beam_search(enc_inputs ,beam_width = 5 ,max_length = 10,start_token = tgt_vocab['<sos>'],end_token = tgt_vocab['<eos>'],):
    gen_sequences = []
    for inputs in enc_inputs:
        inputs = inputs.unsqueeze(0)
        enc_outputs,enc_self_attens = model.encoder(inputs)
        sequences = [[(start_token,1)]]
        for _ in range(max_length):
            all_candidates = []
            for sen in sequences:
                last_token = sen[-1][0]
                dec_inputs = torch.tensor([last_token]).unsqueeze(0)
                dec_outputs,_,_ = model.decoder(dec_inputs,inputs,enc_outputs)
                dec_logits = model.projection(dec_outputs)
                log_probs = torch.log_softmax(dec_logits, dim=-1)
                top_k_probs,top_k_idx = log_probs.topk(k = beam_width,dim = -1)
                for i in range(beam_width):
                    candidate = sen+[(top_k_idx[0,-1,i].item(),sen[-1][1]*top_k_probs[0,-1,i].item())]
                    all_candidates.append(candidate)
            sequences = sorted(all_candidates,key = lambda x:x[-1][1],reverse=True)[:beam_width]
            if any(seq[-1][0] == end_token for seq in sequences):
                break
        best_seq = max(sequences,key = lambda x:x[-1][1])
        final_seq =  [token for token,_ in best_seq]
        final_seq = final_seq[1:]
        gen_sequences.append(final_seq)
    return np.array(gen_sequences)

In [122]:
if __name__ == '__main__':

    model = Transformer()

    criterion = nn.CrossEntropyLoss()
    
    enc_inputs_batchs, dec_input_batchs, target_batchs = make_batch(val_path, src_vocab=src_vocab , tgt_vocab=tgt_vocab , batch_size=5)

    prediction = beam_search(enc_inputs= enc_inputs,max_length=tgt_len+1,beam_size=3)
    
    print(prediction)

[[2, 6647, 319, 755, 7256, 3891, 4218, 7125, 1632, 4306, 7572], [2, 7543, 704, 2743, 4620, 4306, 1134, 1249, 7572, 3943, 3354], [2, 6647, 4827, 5440, 6436, 3932, 1046, 3876, 5946, 3798, 2328], [2, 2952, 6516, 1494, 2049, 4970, 5315, 4306, 2514, 6291, 3603], [2, 7543, 4466, 7396, 2214, 7157, 5546, 7061, 7067, 342, 1148], [2, 4172, 1636, 7312, 1917, 4597, 5946, 2514, 3411, 1799, 53], [2, 4172, 6290, 3854, 7569, 1251, 53, 6128, 121, 7137, 4172], [2, 4172, 4289, 3851, 4306, 3411, 5274, 4466, 7054, 5254, 4844]]


**每层结构变化研究**

```python
enc_inputs = enc_inputs_batchs[0]
print(enc_inputs.shape)

target_batch = target_batch[0]
print(target_batch.shape)
print(enc_inputs_batchs.shape)
enc_inputs_one_batch = enc_inputs_batchs[0]
print(enc_inputs_one_batch.shape)
enc_inputs_one_sen = enc_inputs_one_batch[0]
print(enc_inputs_one_sen.shape)
test_one_batch = enc_inputs_one_sen.unsqueeze(0)
print(test_one_batch.shape)
enc_outputs,enc_self_attns = model.encoder(test_one_batch)
print(enc_outputs.shape)
dec_inputs = torch.full((test_one_batch.shape[0],1),2)
print(dec_inputs.shape)
dec_outputs,_,_= model.decoder(dec_inputs,test_one_batch,enc_outputs)
print(dec_outputs.shape)
dec_logits = model.projection(dec_outputs)
print(dec_logits.shape)
log_probs = torch.log_softmax(dec_logits, dim=-1) # dec_logits: [1, 1, vocab_size]
print(log_probs.shape)
top_k_probs,top_k_idx  =  log_probs.topk(k = 5,dim = -1)
print(top_k_idx.shape)
top_k_probs
top_k_idx[k][-1][i].item(),i
gen_sequence
gen_sequence[-1][0]
for k in range(test_one_batch.shape[0]):
    gen_sequence = [(2,1)]
    print("Now we are processing the ",k,"th sample!")
    last_token = gen_sequence[-1]
    all_candidates = []
    for i in range(5):
        candidate = gen_sequence+[(top_k_idx[k][-1][i].item(),top_k_probs[k][-1][i].item()*gen_sequence[-1][1])]
        all_candidates.append(candidate)
print(all_candidates)
sequences = sorted(all_candidates,key = lambda x: x[-1][1],reverse=True)[:5]
print(sequences)
```